In [18]:
import warnings
warnings.filterwarnings('ignore')

In [19]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter
import tensorflow as tf
from sklearn.preprocessing import StandardScaler,OneHotEncoder
from sklearn.model_selection import train_test_split

In [20]:
cleaned_crime = pd.read_csv('Resources/crime_chip_for_model.csv')
cleaned_crime.head()

,Unnamed: 0,ZIP,Ward,Primary_Type,rankings,Latitude,Longitude,Arrest,Domestic,Population,People/Sq.Mile,National_Rank,state,location,address,chip_latitude,chip_longitude,chipotle,Safety
0,0,60601,27,THEFT,3,NaN,NaN,False,False,"5,591","17,101.15",#271,Illinois,Chicago,"316 N Michigan Ave Chicago, IL 60601 US",41.887288,-87.624848,1,Bad
1,1,60601,27,CRIMINAL DAMAGE,4,41.883932,-87.679964,False,False,"5,591","17,101.15",#271,Illinois,Chicago,"316 N Michigan Ave Chicago, IL 60601 US",41.887288,-87.624848,1,Bad
2,2,60601,27,THEFT,3,41.896569,-87.636063,False,False,"5,591","17,101.15",#271,Illinois,Chicago,"316 N Michigan Ave Chicago, IL 60601 US",41.887288,-87.624848,1,Bad
3,3,60601,27,SEX OFFENSE,3,41.883937,-87.683368,True,False,"5,591","17,101.15",#271,Illinois,Chicago,"316 N Michigan Ave Chicago, IL 60601 US",41.887288,-87.624848,1,Bad
4,4,60601,27,NARCOTICS,4,41.892856,-87.710137,True,False,"5,591","17,101.15",#271,Illinois,Chicago,"316 N Michigan Ave Chicago, IL 60601 US",41.887288,-87.624848,1,Bad


In [21]:
cleaned_crime = cleaned_crime.drop(columns=['Unnamed: 0', 'state', 'location', 'address' ,'chip_latitude', 'chip_longitude'])
cleaned_crime = cleaned_crime.dropna()
cleaned_crime['National_Rank'] = cleaned_crime['National_Rank'].str.replace('#', '')
cleaned_crime['National_Rank'] = cleaned_crime['National_Rank'].str.replace(',', '')
cleaned_crime['National_Rank'] = cleaned_crime['National_Rank'].astype('int')
cleaned_crime['People/Sq.Mile'] = cleaned_crime['People/Sq.Mile'].str.replace(',', '')
cleaned_crime['People/Sq.Mile'] = cleaned_crime['People/Sq.Mile'].astype('float')
cleaned_crime['Population'] = cleaned_crime['Population'].str.replace(',', '')
cleaned_crime['Population'] = cleaned_crime['Population'].astype('int')
cleaned_crime

,ZIP,Ward,Primary_Type,rankings,Latitude,Longitude,Arrest,Domestic,Population,People/Sq.Mile,National_Rank,chipotle,Safety
1,60601,27,CRIMINAL DAMAGE,4,41.883932,-87.679964,False,False,5591,17101.15,271,1,Bad
2,60601,27,THEFT,3,41.896569,-87.636063,False,False,5591,17101.15,271,1,Bad
3,60601,27,SEX OFFENSE,3,41.883937,-87.683368,True,False,5591,17101.15,271,1,Bad
4,60601,27,NARCOTICS,4,41.892856,-87.710137,True,False,5591,17101.15,271,1,Bad
5,60601,27,OFFENSE INVOLVING CHILDREN,7,41.901683,-87.718962,False,True,5591,17101.15,271,1,Bad
...,...,...,...,...,...,...,...,...,...,...,...,...,...
121078,60661,30,ASSAULT,7,41.931096,-87.778958,False,False,4382,15455.28,318,1,Bad
121081,60661,30,OTHER OFFENSE,3,41.941430,-87.737110,False,True,4382,15455.28,318,1,Bad
121083,60661,30,OFFENSE INVOLVING CHILDREN,3,41.940857,-87.728919,False,False,4382,15455.28,318,1,Bad
121084,60661,30,BATTERY,4,41.940049,-87.719873,False,True,4382,15455.28,318,1,Bad


In [24]:
primary_type = cleaned_crime.Primary_Type.value_counts()

In [25]:
# Determine which values to replace if counts are less than ...?
replace_primary = list(primary_type[primary_type < 1000].index)

# Replace in dataframe
for primary in replace_primary:
    cleaned_crime.Primary_Type = cleaned_crime.Primary_Type.replace(primary,"OTHER OFFENSE")
    
# Check to make sure binning was successful
cleaned_crime.Primary_Type.value_counts()

BATTERY                       25230
THEFT                         23517
CRIMINAL DAMAGE               14638
OTHER OFFENSE                 11487
ASSAULT                       10578
DECEPTIVE PRACTICE             7339
BURGLARY                       5386
MOTOR VEHICLE THEFT            5107
NARCOTICS                      4712
WEAPONS VIOLATION              4314
ROBBERY                        4164
CRIMINAL TRESPASS              2646
OFFENSE INVOLVING CHILDREN     1149
Name: Primary_Type, dtype: int64

In [26]:
# Generate our categorical variable lists
crime_obj = list(cleaned_crime.dtypes[cleaned_crime.dtypes == 'object'].index)

In [27]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(cleaned_crime[crime_obj]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(crime_obj)
encode_df.head()

,Primary_Type_ASSAULT,Primary_Type_BATTERY,Primary_Type_BURGLARY,Primary_Type_CRIMINAL DAMAGE,Primary_Type_CRIMINAL TRESPASS,Primary_Type_DECEPTIVE PRACTICE,Primary_Type_MOTOR VEHICLE THEFT,Primary_Type_NARCOTICS,Primary_Type_OFFENSE INVOLVING CHILDREN,Primary_Type_OTHER OFFENSE,Primary_Type_ROBBERY,Primary_Type_THEFT,Primary_Type_WEAPONS VIOLATION,Safety_Bad,Safety_Good
0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0


In [28]:
# Merge one-hot encoded features and drop the originals
cleaned_crime = cleaned_crime.merge(encode_df, left_index=True, right_index=True)
cleaned_crime = cleaned_crime.drop(columns=crime_obj,axis=1)
cleaned_crime.head()

,ZIP,Ward,rankings,Latitude,Longitude,Arrest,Domestic,Population,People/Sq.Mile,National_Rank,...,Primary_Type_DECEPTIVE PRACTICE,Primary_Type_MOTOR VEHICLE THEFT,Primary_Type_NARCOTICS,Primary_Type_OFFENSE INVOLVING CHILDREN,Primary_Type_OTHER OFFENSE,Primary_Type_ROBBERY,Primary_Type_THEFT,Primary_Type_WEAPONS VIOLATION,Safety_Bad,Safety_Good
1,60601,27,4,41.883932,-87.679964,False,False,5591,17101.15,271,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
2,60601,27,3,41.896569,-87.636063,False,False,5591,17101.15,271,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
3,60601,27,3,41.883937,-87.683368,True,False,5591,17101.15,271,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,60601,27,4,41.892856,-87.710137,True,False,5591,17101.15,271,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
5,60601,27,7,41.901683,-87.718962,False,True,5591,17101.15,271,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [29]:
# Split our preprocessed data into our features and target arrays
X = cleaned_crime.drop(columns=['chipotle'],axis=1).values
y = cleaned_crime['chipotle'].values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

# Neural Network

In [30]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [31]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 = 40
hidden_nodes_layer2 = 20

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)


# Second hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu")
)

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 40)                1040      
                                                                 
 dense_4 (Dense)             (None, 20)                820       
                                                                 
 dense_5 (Dense)             (None, 1)                 21        
                                                                 
Total params: 1,881
Trainable params: 1,881
Non-trainable params: 0
_________________________________________________________________


In [32]:
# Import checkpoint dependencies
import os
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the checkpoint path and filenames
os.makedirs("checkpoints/",exist_ok=True)
checkpoint_path = "checkpoints/weights_opt_1.{epoch:02d}.hdf5"

In [33]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Create a callback that saves the model's weights every epoch
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq='epoch',
    period=5
)

# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=100, callbacks=[cp_callback])

Epoch 1/100
2800/2800 [==============================] - 4s 1ms/step - loss: 0.1829 - accuracy: 0.9212
Epoch 2/100
2800/2800 [==============================] - 4s 1ms/step - loss: 0.0056 - accuracy: 0.9994
Epoch 3/100
2800/2800 [==============================] - 4s 1ms/step - loss: 0.0013 - accuracy: 0.9998
Epoch 4/100
2800/2800 [==============================] - 4s 1ms/step - loss: 6.1404e-04 - accuracy: 0.9999
Epoch 5/100
2771/2800 [============================>.] - ETA: 0s - loss: 3.5387e-04 - accuracy: 0.9999
Epoch 5: saving model to checkpoints\weights_opt_1.05.hdf5
2800/2800 [==============================] - 3s 1ms/step - loss: 3.5191e-04 - accuracy: 0.9999
Epoch 6/100
2800/2800 [==============================] - 4s 1ms/step - loss: 1.4860e-04 - accuracy: 1.0000
Epoch 7/100
2800/2800 [==============================] - 4s 2ms/step - loss: 2.4580e-04 - accuracy: 0.9999
Epoch 8/100
2800/2800 [==============================] - 4s 1ms/step - loss: 3.4866e-04 - accuracy: 0.9999
Epoch 

2776/2800 [============================>.] - ETA: 0s - loss: 4.0704e-05 - accuracy: 1.0000
Epoch 60: saving model to checkpoints\weights_opt_1.60.hdf5
2800/2800 [==============================] - 4s 1ms/step - loss: 4.0359e-05 - accuracy: 1.0000
Epoch 61/100
2800/2800 [==============================] - 5s 2ms/step - loss: 2.8506e-08 - accuracy: 1.0000
Epoch 62/100
2800/2800 [==============================] - 4s 1ms/step - loss: 1.3033e-08 - accuracy: 1.0000
Epoch 63/100
2800/2800 [==============================] - 5s 2ms/step - loss: 5.1383e-09 - accuracy: 1.0000
Epoch 64/100
2800/2800 [==============================] - 6s 2ms/step - loss: 2.1012e-09 - accuracy: 1.0000
Epoch 65/100
2794/2800 [============================>.] - ETA: 0s - loss: 1.0680e-09 - accuracy: 1.0000
Epoch 65: saving model to checkpoints\weights_opt_1.65.hdf5
2800/2800 [==============================] - 6s 2ms/step - loss: 1.0671e-09 - accuracy: 1.0000
Epoch 66/100
2800/2800 [==============================] - 4s 2m

In [34]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

934/934 - 1s - loss: 1.4102e-10 - accuracy: 1.0000 - 1s/epoch - 1ms/step
Loss: 1.4102478906874438e-10, Accuracy: 1.0
